In [1]:
import selenium
from selenium import webdriver
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time 
import re

In [ ]:
# Create driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# Get the page url 
page_url = "https://transcripts.foreverdreaming.org/viewtopic.php?f=22&t=4411"
driver.get(page_url) # Open the page 
wait = WebDriverWait(driver, 10) # Wait until the page renders

In [3]:
item_links = [item.get_attribute("href") for item in driver.find_elements(By.CLASS_NAME, 'postlink')] # Get links to the scripts 

In [ ]:
# Scrape scripts from links 
for item_link in item_links[]:
    driver.get(item_link) # Click the link  
    time.sleep(3) 
    l = driver.find_elements(By.TAG_NAME, 'p')
    for i in l: 
        print (i.text) 
